In [2]:
import pandas as pd
import requests
from utils.pandaUtils import convert_dt_cols, get_date_cols

In [3]:
from utils.sqlUtils import connect_db, get_query_from_sql_file
import os
from dotenv import load_dotenv
load_dotenv()

creds = {"driver": os.getenv('DRIVER'),
         "server": os.getenv('SERVER'),
         "db_name": os.getenv('DB_NAME'),
         "db_user": os.getenv('DB_USER'),
         "db_pw": os.getenv('DB_PW')}

conn = connect_db(**creds)
params = { 'card_type': "'S'"}
query = get_query_from_sql_file('test.sql',params)
print(query)
data = pd.read_sql(query, conn)

{'driver': '{ODBC Driver 17 for SQL Server}', 'server': '182.168.1.8', 'db_name': 'SBO_UNICO', 'db_user': 'Unico_RO', 'db_pw': 'Un1c0ro'}
SELECT
    *
FROM
    OCRD
WHERE
    CardType = 'S'


C:\Users\user\AppData\Local\Temp\ipykernel_28068\555187566.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


In [ ]:
req_cols = ["doc_date", "status", "supplier_code", "supplier", "industry_desc",
            "inudstry_name", "trade_suppliers_ind", "payment_terms", "active_vendor", "total"]
file_path = './data/test'
input_file_name = 'active_suppliers_pv_without_columns.csv'
output_file_name = 'active_suppliers_pv.csv'

create_csv_with_cols(file_path, input_file_name, output_file_name, req_cols)

In [ ]:
pv = pd.read_csv("./data/test/active_suppliers_pv.csv",encoding='utf-8')

date_cols = get_date_cols(pv)

for date_col in date_cols:
    pv = correct_date_col(pv, date_col)

In [ ]:
groupby_cols = [pd.Grouper(freq='MS'),'industry_desc','trade_suppliers_ind','active_vendor']
num_cols = ["total"]

pv_ts_index = pv.set_index('doc_date')
grouper_pv = pv_ts_index.groupby(groupby_cols)
pv_ts = grouper_pv[num_cols].sum().reset_index()


In [ ]:
pv_ts[pv_ts['doc_date'] > "2023-09-01"]

,doc_date,industry_desc,trade_suppliers_ind,active_vendor,total
549,2023-12-01,LOCAL SUPPLIERS,OTHER VENDOR,1,3910.0


In [ ]:
pv_ts[pv_ts['doc_date'] >= "2014-01-01"].to_csv('./data/output/active_suppliers_pv.csv',index=False)

In [10]:
outgoing = pd.read_csv("./data/test/ovpm.csv",encoding='utf-8')
incoming = pd.read_csv("./data/test/orct.csv",encoding='utf-8')

date_cols = get_date_cols(outgoing)
outgoing = convert_dt_cols(outgoing, date_cols)
incoming = convert_dt_cols(incoming, date_cols)

C:\Users\user\AppData\Local\Temp\ipykernel_4892\2435511913.py:1: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  outgoing = pd.read_csv("./data/test/ovpm.csv",encoding='utf-8')
C:\Users\user\AppData\Local\Temp\ipykernel_4892\2435511913.py:2: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  incoming = pd.read_csv("./data/test/orct.csv",encoding='utf-8')


In [4]:
purchases = pd.read_csv('purchase_price.csv')

date_cols = get_date_cols(purchases)
purchases = convert_dt_cols(purchases, date_cols)

calculated_purchase_price = pd.pivot_table(purchases, values='price', index='doc_date',
                       columns='pdt_code', aggfunc="sum").ffill()
calculated_purchase_price.columns.name = None
calculated_purchase_price
pdt_purchase_price_ts = calculated_purchase_price.stack(dropna=False).reset_index()
pdt_purchase_price_ts.rename(columns={'doc_date': 'as_of_date', 'level_1':'pdt_code',0:'purchase_price'},inplace=True)
pdt_purchase_price_ts




,as_of_date,pdt_code,purchase_price
0,2023-04-01,BAO001,NaN
1,2023-04-01,BBA001,NaN
2,2023-04-01,BBA001-01,NaN
3,2023-04-01,BBR001,NaN
4,2023-04-01,BBR002,NaN
...,...,...,...
166331,2023-10-01,SWC005-01,6.50
166332,2023-10-01,SWC006,4.00
166333,2023-10-01,SWC006-02,4.50
166334,2023-10-01,SWC008,1.63


In [9]:
date_cols = get_date_cols(purchases)
purchases = convert_dt_cols(purchases, date_cols)

purchases.rename(columns={'doc_date': 'as_of_date',
                 'price': 'purchase_price'},inplace=True)
latest_purchase_price = pdt_purchase_price_ts[pdt_purchase_price_ts['as_of_date'] == '2023-10-01']

with_latest_purchases = pd.concat([latest_purchase_price,purchases],ignore_index=True)

calculated_purchase_price = pd.pivot_table(with_latest_purchases, values='purchase_price', index='as_of_date',
                       columns='pdt_code', aggfunc="sum").ffill()
calculated_purchase_price.columns.name = None
pdt_purchase_price_ts = calculated_purchase_price.stack().reset_index()
pdt_purchase_price_ts.rename(columns={'level_1':'pdt_code',0:'purchase_price'},inplace=True)


,as_of_date,pdt_code,purchase_price
0,2023-10-01,BAO001,6.50
1,2023-10-01,BBA001,9.00
2,2023-10-01,BBA001-01,10.80
3,2023-10-01,BBR001,6.70
4,2023-10-01,BBR002,7.00
...,...,...,...
7943,2023-10-01,DUCK20,0.92
7944,2023-10-01,DUCK06,2.00
7945,2023-10-01,PORO01,20.00
7946,2023-10-01,FRS003,24.00


In [ ]:
roupby_cols = [pd.Grouper(freq='MS')]
num_cols = ["cash_sum", "credit_sum", "check_sum",
            "transfer_sum", "doc_total"]

outgoing_ts_index = outgoing.set_index('doc_date')
grouper_outgoing = outgoing_ts_index.groupby(groupby_cols)
outgoing_sum = grouper_outgoing[num_cols].sum().reset_index()

incoming_ts_index = incoming.set_index('doc_date')
grouper_incoming = incoming_ts_index.groupby(groupby_cols)
incoming_sum = grouper_incoming[num_cols].sum().reset_index()

In [ ]:
incoming_sum_long = pd.melt(incoming_sum, id_vars=['doc_date'], value_vars=num_cols, var_name = 'payment_type', value_name='total')
outgoing_sum_long = pd.melt(outgoing_sum, id_vars=['doc_date'], value_vars=num_cols, var_name = 'payment_type', value_name='total')

incoming_sum_long[incoming_sum_long['payment_type'] != 'doc_total'].to_csv('./data/output/monthly_incoming_by_type.csv',index=False)
outgoing_sum_long[outgoing_sum_long['payment_type'] != 'doc_total'].to_csv('./data/output/monthly_outgoing_by_type.csv',index=False)

In [ ]:
payments_ts = outgoing_sum[['doc_date','doc_total']].rename(columns={'doc_total':'outgoing'}).merge(incoming_sum[['doc_date','doc_total']].rename(columns={'doc_total':'incoming'}))
payments_ts['diff'] = payments_ts['incoming'] - payments_ts['outgoing']
payments_ts['outgoing'] = payments_ts['outgoing'] * -1
payments_ts.to_csv('./data/output/monthly_payments_ts.csv',index=False)

In [ ]:
ordr = pd.read_csv("./data/test/ordr.csv",encoding='utf-8')
rdr1 = pd.read_csv("./data/test/rdr1_short.csv",
                   encoding='utf-8')

In [ ]:
ordr_date_cols = get_date_cols(ordr)
rdr1_date_cols = get_date_cols(rdr1)

for col in ordr_date_cols:
    ordr = correct_date_col(ordr, col)

for col in rdr1_date_cols:
    rdr1 = correct_date_col(rdr1, col)

In [ ]:
rdr1['ordr_doc_no'] = rdr1['ordr_doc_no'].fillna(-1)
rdr1['ordr_doc_no'] = rdr1['ordr_doc_no'].astype('int64')

In [ ]:
# sales since 2022
start_date = '2014-01-01'
end_date = '2023-08-02'
date_col = 'doc_date'
req_cols = ['doc_date','ordr_doc_no','sales_agent','item_code','item_description','total_before_vat','customer_name']
sales = filter_df_by_date_range(rdr1, date_col, start_date, end_date)[req_cols]

In [ ]:
groupby_cols = ['sales_agent','doc_date','ordr_doc_no']
sales_agg_to_doc = sales.groupby(groupby_cols)['total_before_vat'].sum().reset_index()


In [ ]:
req_cols = ['doc_no','vat_total','total']
ordr_filtered = filter_df_by_date_range(ordr, date_col, start_date, end_date)[req_cols]
ordr_filtered['total_before_vat_ordr'] = ordr_filtered['total'] - ordr_filtered['vat_total']

In [ ]:
check_df = sales_agg_to_doc.merge(ordr_filtered[['doc_no','total_before_vat_ordr']], left_on='ordr_doc_no', right_on='doc_no')

In [ ]:
groupby_cols = [pd.Grouper(freq='MS'),'sales_agent']
sales_by_month_year = sales_agg_to_doc.set_index('doc_date')

grouper_sales_by_month_year = sales_by_month_year.groupby(groupby_cols)
sales_by_month_year = grouper_sales_by_month_year['total_before_vat'].sum().reset_index()


In [ ]:
sales_by_month_year.to_csv('./data/output/sales_agent_sales_ts.csv',index=False)

Timestamp('2023-08-02 00:00:00')

# Processed Item Analysis

In [ ]:
processed_items = pd.read_csv("./data/test/unique_item_codes_in_goods_receipt_corrected.csv")
processed_items.dropna(inplace=True)
processed_items['Processed'] = processed_items['Processed '].str.lower()
processed_items.drop(columns=["Processed "],inplace=True)
processed_items_count = processed_items.groupby(['Processed'])['ItemCode'].count().reset_index()
processed_items_count.rename(columns={'ItemCode': 'item_count', 'Processed': "processed"},inplace=True)
processed_items_list = list(processed_items.loc[processed_items['Processed'] == 'y', 'ItemCode'])

In [ ]:
date_col = 'doc_date'
sales['processed'] = sales['item_code'].isin(processed_items_list)
req_cols = ['processed','doc_date','total_before_vat']

sales_by_processed = sales[req_cols] 

groupby_cols = [pd.Grouper(freq='MS'),'processed']
sales_by_processed = sales_by_processed.set_index('doc_date')

grouper_sales_by_processed = sales_by_processed.groupby(groupby_cols)
sales_by_processed = grouper_sales_by_processed['total_before_vat'].sum().reset_index()

In [ ]:
sales_by_processed.to_csv('./data/output/sales_by_processed_ts.csv',index=False)


In [ ]:
incoming = pd.read_csv("./data/raw/incoming_payments.csv")
outgoing = pd.read_csv("./data/raw/outgoing_payments.csv")

date_col = 'Posting Date'
incoming = correct_date_col(incoming, date_col,format="%d.%m.%Y")
outgoing = correct_date_col(outgoing, date_col,format="%d.%m.%Y")

In [ ]:
incoming['amount']

,#,Document Number,Posting Date,Customer/Vendor Code,Customer/Vendor Name,Customer/Vendor Name.1,Document Currency,Check Amount,Transfer Amount,Cash Amount,Reference 1,Reference 2,Counter Reference,Remarks,Journal Remarks,Journal Remarks.1
0,1,361645,2023-01-04,UCF0036,FENG ZHEN LOR MEE,FENG ZHEN LOR MEE,SGD,0,0,251.1,36-1645,NaN,NaN,3/1 23100695,3/1 23100695,3/1 23100695
1,2,361646,2023-01-04,UCS0228,SHU WEI YUAN FOOD,SHU WEI YUAN FOOD,SGD,0,0,163,36-1646,NaN,NaN,3-Jan,3-Jan,3-Jan
2,3,361648,2023-01-03,UCN0015,365 FOOD SERVICES PTE LTD,365 FOOD SERVICES PTE LTD,SGD,0,"1,835.85",0,36-1648,NaN,NaN,29/12-3/1 B.TRANS,29/12-3/1 B.TRANS,29/12-3/1 B.TRANS
3,4,361649,2023-01-04,UCD0025,D AND S MINIMART,D AND S MINIMART,SGD,0,0,234.9,36-1649,NaN,NaN,3-Jan,3-Jan,3-Jan
4,5,361650,2023-01-03,UCN0053,365 FOOD SERVICES PTE LTD,365 FOOD SERVICES PTE LTD,SGD,0,443.16,0,36-1650,NaN,NaN,29/12-3/1 B.TRANS,29/12-3/1 B.TRANS,29/12-3/1 B.TRANS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22308,22309,370273,2023-03-28,UCF0086,FISH SOUP,FISH SOUP,SGD,0,0,77.7,37-0273,NaN,NaN,27-Mar,27-Mar,27-Mar
22309,22310,370274,2023-03-27,UCN0122,NANYANG TRADITIONAL FOOD PTE LTD,NANYANG TRADITIONAL FOOD PTE LTD,SGD,0,"9,640.96",0,37-0274,NaN,NaN,SEPT-MAR B.TRANS,SEPT-MAR B.TRANS,SEPT-MAR B.TRANS
22310,22311,370275,2023-03-28,UCM0298,MR YAN QIAILONG,MR YAN QIAILONG,SGD,0,0,39.9,37-0275,NaN,NaN,28-Mar,28-Mar,28-Mar
22311,22312,370276,2023-03-27,UCP0142,PRO HOLDINGS PTE LTD,PRO HOLDINGS PTE LTD,SGD,0,206.06,0,37-0276,NaN,NaN,23/3 B.TRANS 23131232,23/3 B.TRANS 23131232,23/3 B.TRANS 23131232


In [ ]:
BASE_URL = "https://open-api.tiktokglobalshop.com"
endpoint = "api/products/search"
access_token = "ROW_3QuadgAAAADYU_GBsCFMHukqC8Lf7GRGOWlE4gTXauOLEjhons-K3ygHBLr4fI5b0JswlQP-vbB97mb6ZOGnR2v1hBd35pVw-0dmIrJjf-KbF-HQskg039pd3rF8CLPWs077nj0pSkbkgHfUXYfX803kiusbJwLyanSyPNSq_EI-2maKoDEknQ"
app_key = "69gf9q07bp28l"
data = {"page_number": 1, "page_size": 100}
params = {"access_token": access_token, "app_key": app_key }
headers = {'Content-Type': 'application/json'}
response = requests.request(
    "POST", url=f"{BASE_URL}/{endpoint}", params=params, headers=headers, data=data)

In [ ]:
response.text

'{"code":106001,"message":"signature is invalid","request_id":"20230728070517A25CF9869D22B7003821"}'

In [ ]:

BASE_URL = "https://developers.onemap.sg/"

def searchPostal(postal_code):

    endpoint = "commonapi/search"
    headers = {'cache-control': 'no-cache, max-age=0',
               'content-type': 'application/json'}
    params = {"searchVal": postal_code, "returnGeom": "N",
              "getAddrDetails": "Y", "pageNum": 1}

    result = requests.get(url=f"{BASE_URL}/{endpoint}",
                          params=params, headers=headers)
    
    return result

def formatSearchPostal(result, postal_code):
    
    result_list = result.json()['results']
    returned_address = pd.DataFrame.from_dict(result_list[0], orient="index").T
    returned_address['INPUT_POSTAL'] = postal_code
    returned_address['QUERY_STATUS'] = "SUCCESS"
    
    return returned_address


In [ ]:
postal_inputs

,Zipcode,Detail Address,Unit/floor,Additional address information
0,681166,166A Teck Whye Crescent,#18-355\t,NaN
1,540254,254 Compassvale Road,10-706\t,deliver 12pm
2,600273,273 Toh Guan Road,#02-55\t,NaN
3,531471,471A Upper Serangoon Crescent,#10-386\t,NaN
4,828746,"Waterwoods, 15 Punggol Field Walk",17-04\t,NaN
...,...,...,...,...
115,520298,Blk 298 Tampines St 22,#08-554\t,Call when reaching
116,427711,"Parc Seabreeze, 532 Joo Chiat Road",07-04\t,NaN
117,823268,268C Punggol Field,#03-159\t,NaN
118,438774,"Haig Residences, 163 Haig Road",#02-03\t,NaN


In [ ]:

file="address_list.xlsx"
postal_inputs = pd.read_excel(file)

returned_addresses = []

for postal_code in postal_inputs['Zipcode']:
    print(postal_code)

    api_result = searchPostal(postal_code)

    if api_result.ok:
        result_dict = api_result.json()

    if len(result_dict["results"]) != 0:
        returned_address = formatSearchPostal(api_result, postal_code)
    else:
        returned_address = pd.DataFrame.from_dict({"BLK_NO": "", "ROAD_NAME": "", "BUILDING": "", "ADDRESS": "",
                                                  "POSTAL": "", "INPUT_POSTAL": postal_code, "QUERY_STATUS": "NO_RESULTS"}, orient="index").T
    
    returned_addresses.append(returned_address)


681166
{'found': 1, 'totalNumPages': 1, 'pageNum': 1, 'results': [{'SEARCHVAL': 'TECK WHYE CREST', 'BLK_NO': '166A', 'ROAD_NAME': 'TECK WHYE CRESCENT', 'BUILDING': 'TECK WHYE CREST', 'ADDRESS': '166A TECK WHYE CRESCENT TECK WHYE CREST SINGAPORE 681166', 'POSTAL': '681166'}]}
{'found': 1, 'totalNumPages': 1, 'pageNum': 1, 'results': [{'SEARCHVAL': 'TECK WHYE CREST', 'BLK_NO': '166A', 'ROAD_NAME': 'TECK WHYE CRESCENT', 'BUILDING': 'TECK WHYE CREST', 'ADDRESS': '166A TECK WHYE CRESCENT TECK WHYE CREST SINGAPORE 681166', 'POSTAL': '681166'}]}
540254
{'found': 1, 'totalNumPages': 1, 'pageNum': 1, 'results': [{'SEARCHVAL': 'COMPASSVALE HAVEN', 'BLK_NO': '254', 'ROAD_NAME': 'COMPASSVALE ROAD', 'BUILDING': 'COMPASSVALE HAVEN', 'ADDRESS': '254 COMPASSVALE ROAD COMPASSVALE HAVEN SINGAPORE 540254', 'POSTAL': '540254'}]}
{'found': 1, 'totalNumPages': 1, 'pageNum': 1, 'results': [{'SEARCHVAL': 'COMPASSVALE HAVEN', 'BLK_NO': '254', 'ROAD_NAME': 'COMPASSVALE ROAD', 'BUILDING': 'COMPASSVALE HAVEN', 'A

In [ ]:
found_addresses = pd.concat(returned_addresses,ignore_index=True)
output = postal_inputs.merge(found_addresses, left_on="Zipcode", right_on='INPUT_POSTAL', how='left')
output.to_csv('returned_addresses.csv', index=False)


In [ ]:
output[output['QUERY_STATUS'] != 'SUCCESS']

,Zipcode,Detail Address,Unit/floor,Additional address information,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,INPUT_POSTAL,QUERY_STATUS
84,409289,810 Geylang Road,04-27 City Plaza\t,NaN,NaN,,,,,,409289,NO_RESULTS


In [ ]:
output[output['QUERY_STATUS'] != 'SUCCESS']

,Zipcode,Detail Address,Unit/floor,Additional address information,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,INPUT_POSTAL,QUERY_STATUS
84,409289,810 Geylang Road,04-27 City Plaza\t,NaN,NaN,,,,,,409289,NO_RESULTS


In [ ]:
postal_code = 409289
test = searchPostal(postal_code)


In [ ]:
test.json()

{'found': 0, 'totalNumPages': 0, 'pageNum': 1, 'results': []}

In [ ]:
no_results = pd.DataFrame.from_dict({"BLK_NO": "","ROAD_NAME": "","BUILDING":"","ADDRESS":"","POSTAL":"", "INPUT_POSTAL":postal_code},orient="index").T
